In [1]:
import pandas as pd
import numpy as np
import nltk
import pickle
import re
import pymorphy2
from numpy import hstack
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)

Смотрим на данные

In [2]:
ctree=pd.read_csv('categories_tree.csv')
ctree.head(10)

,id,title,parent_id
0,1,Все категории,0
1,114,Урбеч,1913
2,115,Варенье и джемы,328
3,128,Сухие завтраки,2475
4,131,Масла,2475
5,132,Супы,2475
6,133,Соусы,2475
7,134,Приправы,14283
8,135,"Сахар, соль",2475
9,136,Мука,2475


In [3]:
df=pd.read_parquet('train.parquet', engine='pyarrow')
df.head()

,id,title,short_description,name_value_characteristics,rating,feedback_quantity,category_id
0,1267423,Muhle Manikure Песочные колпачки для педикюра 1015 PW Средний (150 грит) (упаковка 10 шт),Muhle Manikure Колпачок песочный шлифовальный для педикюра 1015 PW Белый Средний (150 грит) (упаковка 10 шт),None,0.000000,0,2693
1,128833,"Sony Xperia L1 Защитное стекло 2,5D",None,None,4.666667,9,13408
2,569924,"Конверт для денег Прекрасная роза, 16,5 х 8 см","Конверт для денег «Прекрасная роза», 16,5 × 8 см",None,5.000000,6,11790
3,1264824,Серьги,None,None,0.000000,0,14076
4,1339052,Наклейки на унитаз для туалета на крышку бачок на дверь ванную на стену мебель зеркало,"Водостойкая, интересная наклейка на унитаз размер 17х22 см",None,0.000000,0,12401


In [4]:
dftest=pd.read_parquet('test.parquet', engine='pyarrow')
dftest.head()

,id,title,short_description,name_value_characteristics,rating,feedback_quantity
0,1070974,Браслет из натуральных камней LOTUS,None,None,0.000000,0
1,450413,Fusion Life - Шампунь для сухих и окрашенных волос Личи 460 мл,None,None,4.333333,6
2,126857,"Микрофон для ПК jack 3,5мм всенаправленный","универсальный 3,5 мм микрофон запишет ваш звук",None,3.708333,24
3,1577569,Серьги гвоздики сердце,Серьги гвоздики сердце,None,0.000000,0
4,869328,"Чёрно-красная стильная брошь ""Тюльпаны"" из акрила, брошь-булавка ручной работы",Стильная и яркая брошь ручной работы! Великолепный подарок для вас и ваших близких!,None,0.000000,0


In [18]:
df.describe()

,id,rating,feedback_quantity,category_id
count,2.834520e+05,283452.000000,283452.00000,283452.000000
mean,9.401805e+05,1.814641,4.34656,12244.567295
std,4.191303e+05,2.309150,34.67245,2543.823093
min,9.500000e+01,0.000000,-1.00000,2598.000000
25%,6.063808e+05,0.000000,0.00000,12049.000000
50%,9.963495e+05,0.000000,0.00000,12730.000000
75%,1.298164e+06,4.871795,2.00000,13408.000000
max,1.655762e+06,5.000000,6102.00000,14559.000000


In [19]:
df.category_id.unique().size

1231

преобразовываем данные

In [5]:
#заменяем все нулевые значения на точки, чтобы сложить дальше столбцы со словами
df.short_description.fillna(value='.', inplace = True)
df.name_value_characteristics.fillna(value='.', inplace = True)

In [6]:
#складываем столбцы с текстом, приводим к нижнему регистру и убираем все специальные знаки и цифры
df['words'] = df.title+' '+df.short_description#+' '+df.name_value_characteristics
df['words']=df.words.str.lower()
df['words'] = df['words'].str.replace('\W|\d', ' ')

C:\Users\kiril\AppData\Local\Temp/ipykernel_7908/2953944529.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  df['words'] = df['words'].str.replace('\W|\d', ' ')


In [20]:
#чтобы увидеть распределение слов, делаем словарь из всех слов(и их количества)
lstlst=df['words'].str.split().tolist()
lst = [x for l in lstlst for x in l]
from collections import Counter
amount=Counter(lst)

In [21]:
onlyone=[]
n=0
dict(amount)
for i in amount:
    n+=1
    if amount[i]==1:
        onlyone.append(i)
print(n)

86863


In [7]:
def pos(word, morth=pymorphy2.MorphAnalyzer()):
    return morth.parse(word)[0].tag.POS

functors_pos = {'INTJ', 'PRCL', 'CONJ', 'PREP'}  # function words

In [8]:
  #1.в словах длиннее 7 символов удаляем последнюю букву(или две),чтобы окончания не влияли на выбор слов алгоритмом
  #2.убираем самые используемые слова,не несущие смысловой нагрузки
#3.удаляем служебные части речи
#4.убираем все слова короче двух букв,так как в них нет смысла
  #5.удаляем повторяющиеся слова в одном товаре
#nosensewords=['для','это','без','или','виде','при','любой','none']#,'ваш','вашего','отлично','февраля','сыворотка','душа','очень','надписью','век','лет','год','так','что','можно']
for index, row in df.iterrows():
    new_words=[]
    for word in row['words'].split():
        '''if len(word)>=7:
            new_words.append(word[:-1])'''
        '''if word in nosensewords:
            continue'''
        if pos(word) not in functors_pos:
            new_words.append(word)
        if len(word)>2:
            new_words.append(word)
    '''new_words = list(set(new_words))'''
    df.at[index,'words']=' '.join(new_words)

In [9]:
#Также добавляем данные, которые были удалены. Что позволяет усилить основные данные и при этом учитывать те,которые стирались
df['words'] =df.words+' '+df.title#+' '+df.name_value_characteristics#+' '+df.short_description#
df['words']=df.words.str.lower()
df['words'] = df['words'].str.replace('\W|\d', ' ')

C:\Users\kiril\AppData\Local\Temp/ipykernel_7908/1966941030.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  df['words'] = df['words'].str.replace('\W|\d', ' ')


1.среди всех слов используемых в названиях, ~35-40% употребляются по одному разу.
если поверхностно их просмотреть, то сразу видно, что в большинстве из них просто ошибки, и если установить какую нибудь бибиотеку, которая будет их исправлять, это позволит значительно повысить эффективность модели
2.есть множество слов, которые не несут смысловой нагрузки,и если их убрать,процент значительно повышается. Но проверять их приходится вручную. Самые очевидные были убраны из текста.
В будущем это можно автоматизировать, так как каждое такое слово дает 0.02-0.05% к результату

In [22]:
df.head()

,id,title,short_description,name_value_characteristics,rating,feedback_quantity,category_id,words
0,1267423,Muhle Manikure Песочные колпачки для педикюра 1015 PW Средний (150 грит) (упаковка 10 шт),Muhle Manikure Колпачок песочный шлифовальный для педикюра 1015 PW Белый Средний (150 грит) (упаковка 10 шт),.,0.000000,0,2693,muhle muhle manikure manikure песочные песочные колпачки колпачки для педикюра педикюра pw средний средний грит грит упаковка упаковка шт muhle muhle manikure manikure колпачок колпачок песочный песочный шлифовальный шлифовальный для педикюра педикюра pw белый белый средний средний грит грит упаковка упаковка шт muhle manikure песочные колпачки для педикюра pw средний грит упаковка шт
1,128833,"Sony Xperia L1 Защитное стекло 2,5D",.,.,4.666667,9,13408,sony sony xperia xperia l защитное защитное стекло стекло d sony xperia l защитное стекло d
2,569924,"Конверт для денег Прекрасная роза, 16,5 х 8 см","Конверт для денег «Прекрасная роза», 16,5 × 8 см",.,5.000000,6,11790,конверт конверт для денег денег прекрасная прекрасная роза роза х см конверт конверт для денег денег прекрасная прекрасная роза роза см конверт для денег прекрасная роза х см
3,1264824,Серьги,.,.,0.000000,0,14076,серьги серьги серьги
4,1339052,Наклейки на унитаз для туалета на крышку бачок на дверь ванную на стену мебель зеркало,"Водостойкая, интересная наклейка на унитаз размер 17х22 см",.,0.000000,0,12401,наклейки наклейки унитаз унитаз для туалета туалета крышку крышку бачок бачок дверь дверь ванную ванную стену стену мебель мебель зеркало зеркало водостойкая водостойкая интересная интересная наклейка наклейка унитаз унитаз размер размер х см наклейки на унитаз для туалета на крышку бачок на дверь ванную на стену мебель зеркало


In [24]:
X_train, X_test, y_train, y_test = train_test_split(df['words'], df['category_id'], test_size=0.2, random_state=1)

Делаем модель

In [25]:
vec = CountVectorizer()
X_train = vec.fit_transform(X_train)
X_test = vec.transform(X_test)
# Инициализировать наивную байесовскую модель с конфигурацией по умолчанию
mnb = MultinomialNB()
# Используем данные обучения для оценки параметров модели и прогнозирования
mnb.fit(X_train, y_train)
y_predict = mnb.predict(X_test)

# Результаты прогнозирования выхода
print('The Accuracy of Naive Bayes:', mnb.score(X_test, y_test)*100)

The Accuracy of Naive Bayes: 78.33518547917659


построим еще модель, для классификации

In [ ]:
features=['rating','feedback_quantity']
X=df[features]
X_train, X_test, y_train, y_test = train_test_split(X, df['category_id'], test_size=0.2, random_state=1)

In [ ]:
vec = CountVectorizer()
X_train = vec.fit_transform(X_train)
X_test = vec.transform(X_test)

In [ ]:
classifier = RandomForestClassifier(n_estimators=1000, random_state=0)
classifier.fit(X_train, y_train) 

In [ ]:
print('The Accuracy of RandomForest:', classifier.score(X_test, y_test)*100)

размечаем тестовые данные

In [11]:
dftest.short_description.fillna(value='.', inplace = True)
dftest.name_value_characteristics.fillna(value='.', inplace = True)
dftest['words'] = dftest.title+' '+dftest.short_description
dftest['words'] = dftest.words.str.lower()
dftest['words'] = dftest['words'].str.replace('\W|\d', ' ')
def pos(word, morth=pymorphy2.MorphAnalyzer()):
    return morth.parse(word)[0].tag.POS

functors_pos = {'INTJ', 'PRCL', 'CONJ', 'PREP'} 

C:\Users\kiril\AppData\Local\Temp/ipykernel_7908/631749248.py:5: FutureWarning: The default value of regex will change from True to False in a future version.
  dftest['words'] = dftest['words'].str.replace('\W|\d', ' ')


In [12]:
for index, row in dftest.iterrows():
    new_words=[]
    for word in row['words'].split():
        if pos(word) not in functors_pos:
            new_words.append(word)
        if len(word)>2:
            new_words.append(word)
    dftest.at[index,'words']=' '.join(new_words)
dftest['words'] = dftest.words+' '+dftest.title
dftest['words'] = dftest.words.str.lower()
dftest['words'] = dftest['words'].str.replace('\W|\d', ' ')

C:\Users\kiril\AppData\Local\Temp/ipykernel_7908/3293239379.py:11: FutureWarning: The default value of regex will change from True to False in a future version.
  dftest['words'] = dftest['words'].str.replace('\W|\d', ' ')


In [13]:
X_train=df['words']
X_test=dftest['words']
y_train=df['category_id']
vec = CountVectorizer()
X_train = vec.fit_transform(X_train)
X_test = vec.transform(X_test)
mnb = MultinomialNB()
mnb.fit(X_train, y_train)
y_predict = mnb.predict(X_test)

In [17]:
dfresult = pd.DataFrame(data={'id': dftest['id'], 'predicted_category_id': y_predict})
dfresult.to_parquet('result.parquet') 

Вывод: Сделал классификацию на основе наивного баессовского классификатора, так как этот классификатор лучше всего работает с текстом.
Сложил текстовые данные из столбцов(не считая характеристик), и дальше работал с этим текстом.
Удалил все специальные знаки и цифры, удалил все служебные части речи и слова короче двух букв(в основном остались после удаления цифр.
Добавил данные в которых все таки есть служебные части речи и слова короче двух букв, чтобы они учитывались, но в меньшей степени, так как остальные слова встречаются несколько раз.
На тренировочной выборке(0,2) получили результат 78,3% без учета иерархии.

Результат можно улучшить:
1.используя другие модели и ансамбль из них
2.дальнейшее улучшение данных
    а)добавив библиотеку исправляющую орфографические ошибки
    б)добавить лемматизацию слов
    в)исключение слов не несущих смысловой нагрузки(скидка,бесплатно,новый...)
3.учет оставшихся столбцов и работа над улучшением качества в них
4.учет фотографий товара
5.учет цены товара
6.при решении учитывать дерево категорий